# Some general registrations

In [6]:
import SimpleITK as sitk
import os
from tqdm import tqdm
import sys
sys.path.append("..")
from config import *
from utils import generate_BIDS_path

In [2]:
TO_REGISTER = CONTRASTS["FLAIR_ORIGINAL"]
REGISTERED = { "acquisition": None, "suffix": "FLAIR_2", "scope": "segmentation_probability_maps", "extension": "nii.gz"}
INTERPOLATION_ORDER = 3

TO_REGISTER_EXTRA = ({ "acquisition": None, "suffix": "probabilitiesmap", "scope": "segmentation_probability_maps", "extension": "nii.gz" }, )
REGISTERED_EXTRA = (CONTRASTS["PMAP"], )
INTERPOLATION_ORDERS = (3, )

REGISTRATION_SPACE = CONTRASTS["FLAIR"]
TO_REPLACE = False

KeyError: 'FLAIR_ORIGINAL'

In [3]:
TO_REGISTER = CONTRASTS["3T_MP2RAGE_UNI"]
REGISTERED = CONTRASTS["7TREG_MP2RAGE_UNI"]
INTERPOLATION_ORDER = 3

TO_REGISTER_EXTRA = (CONTRASTS["3T_FLAIR"], CONTRASTS["3T_SEGMENTATION"])
REGISTERED_EXTRA = (CONTRASTS["7TREG_FLAIR"], CONTRASTS["SEGMENTATION"])
INTERPOLATION_ORDERS = (3, 0)

REGISTRATION_SPACE = CONTRASTS["MP2RAGE_UNI_ORIGINAL_SK"]
TO_REPLACE = False

In [9]:
%%time

for db_id in (DATASET_NIH7T_ID, ):#range(len(AVAILABLE_DATASETS)):
    dataset = AVAILABLE_DATASETS[db_id]
    for subject in tqdm(dataset.get_subjects()):
        moving_path = generate_BIDS_path(db_id, **TO_REGISTER, subject=subject)
        if REGISTERED is not None:
            moving_path_to = generate_BIDS_path(db_id, **REGISTERED, subject=subject)
        moving_paths_extra = [generate_BIDS_path(db_id, **vals, subject=subject) for vals in TO_REGISTER_EXTRA]
        moving_paths_extra_to = [generate_BIDS_path(db_id, **vals, subject=subject) for vals in REGISTERED_EXTRA]
        static_path = generate_BIDS_path(db_id, **REGISTRATION_SPACE, subject=subject)
        
        if not os.path.exists(moving_path):
            print(f"[{subject}] Moving file not found.")
            continue
        if not TO_REPLACE and REGISTERED is not None and os.path.exists(moving_path_to):
            print(f"[{subject}] Registered file already exists.")
            continue
        if not os.path.exists(static_path):
            print(f"[{subject}] Static file not found.")
            continue
            
        all_ok = True
        for (pfrom, pto) in zip(moving_paths_extra, moving_paths_extra_to):
            if not os.path.exists(pfrom):
                print(f"[{subject}] Moving extra file not found.")
                all_ok = False
            if not TO_REPLACE and os.path.exists(pto):
                print(f"[{subject}] Registered extra file already exists.")
                all_ok = False
        if not all_ok:
            continue
                
        movingImage = sitk.ReadImage(moving_path)
        staticImage = sitk.ReadImage(static_path)
        
        parameterMap = sitk.GetDefaultParameterMap('rigid')
        parameterMap["MaximumNumberOfIterations"] = ("5000", )
        parameterMap["NumberOfResolutions"] = ("6", )
        parameterMap['FinalBSplineInterpolationOrder'] = [str(INTERPOLATION_ORDER)]
        
        elastixImageFilter = sitk.ElastixImageFilter()
        elastixImageFilter.SetFixedImage(staticImage)
        elastixImageFilter.SetMovingImage(movingImage)
        elastixImageFilter.SetParameterMap(parameterMap)
        elastixImageFilter.Execute()

        resultImage = elastixImageFilter.GetResultImage()
        if REGISTERED is not None:
            sitk.WriteImage(resultImage, moving_path_to)

        for (pfrom, pto, interpolation_order) in zip(moving_paths_extra, moving_paths_extra_to, INTERPOLATION_ORDERS):  
            transformParameterMap = elastixImageFilter.GetTransformParameterMap()
            for tr in transformParameterMap:
                tr['FinalBSplineInterpolationOrder'] = [str(interpolation_order), ]
            transformixImageFilter = sitk.TransformixImageFilter()
            transformixImageFilter.SetTransformParameterMap(transformParameterMap)
            
            movingImage = sitk.ReadImage(pfrom)
            transformixImageFilter.SetMovingImage(movingImage)
            transformixImageFilter.Execute()
            resultImage = transformixImageFilter.GetResultImage()
            sitk.WriteImage(resultImage, pto)
        continue
    

  0%|          | 0/20 [00:03<?, ?it/s]


RuntimeError: Exception thrown in SimpleITK ElastixImageFilter_Execute: /home/german/simpleelastix/SimpleElastix/Code/Elastix/src/sitkElastixImageFilterImpl.cxx:259:
sitk::ERROR: 
itk::ExceptionObject (0x55bd9546a520)
Location: "unknown" 
File: /home/german/simpleelastix/build/Elastix/Core/Main/elxElastixFilter.hxx
Line: 253
Description: itk::ERROR: Self(0x55bd95c89390): Internal elastix error: See elastix log (use LogToConsoleOn() or LogToFileOn()).



In [ ]:
# TO FIX AFFINE MATRICES

import sys
sys.path.append("..")
import nibabel as nib
from config import *


for patient, dataset in ((12, DATASET_CHUV), (39, DATASET_CHUV), (41, DATASET_CHUV), ):
    t2star_path = dataset.get(return_type="filename", subject=f"{patient:03d}", **CONTRASTS["T2STAR_MAG"])[0]
    flairstar_path = dataset.get(return_type="filename", subject=f"{patient:03d}", **CONTRASTS["FLAIRSTAR"])[0]
    flair_path = dataset.get(return_type="filename", subject=f"{patient:03d}", **CONTRASTS["FLAIR"])[0]
    phase_path = dataset.get(return_type="filename", subject=f"{patient:03d}", **CONTRASTS["T2STAR_PHASE"])[0]
    
    output_path = t2star_path.replace(".nii.gz", "_fixed.nii.gz")
    #if os.path.exists(output_path):
        #continue
    t2 = nib.load(t2star_path)
    
    phase = nib.load(phase_path)
    #flairstar = nib.load(os.path.join(subdir, flairstar_name))
    #print(t2.affine)
    
    new_image = nib.Nifti1Image(t2.get_data(), phase.affine, t2.header)
    nib.save(new_image, output_path)

In [3]:
# TO FIX PMAPS

import sys
sys.path.append("..")
import nibabel as nib
from config import *

dataset = DATASET_NIH7T
for patient in dataset.get_subjects():
    #pmap_reg = dataset.get(return_type="filename", subject=f"{patient}", **CONTRASTS["PMAP"])
    #if len(pmap_reg) == 1:
        #continue
    print(f"Fixing patient {patient}...")
    pmap_path = dataset.get(return_type="filename", subject=f"{patient}", **{ "acquisition": None, "suffix": "probabilitiesmap", "scope": "segmentation_probability_maps", "extension": "nii.gz" })
    if len(pmap_path) == 0:
        print(f"PROBLEM: {patient}")
        continue
    pmap_path = pmap_path[0]
    
    im = nib.load(pmap_path)
    image = im.get_fdata()[:,:,:,:,1].squeeze()
    
    new_image = nib.Nifti1Image(image, im.affine, im.header)
    nib.save(new_image, pmap_path)

Fixing patient 001...
Fixing patient 002...
Fixing patient 003...
Fixing patient 004...
Fixing patient 005...
Fixing patient 006...
Fixing patient 007...
Fixing patient 008...
Fixing patient 009...
Fixing patient 010...
Fixing patient 011...
Fixing patient 012...
Fixing patient 013...
Fixing patient 014...
Fixing patient 015...
Fixing patient 016...
Fixing patient 017...
Fixing patient 018...
Fixing patient 019...
Fixing patient 020...
